# MiADE (Medical information AI Data Extractor) Tutorial

## Installation and downloads

### Install MiADE

In [1]:
%pip install miade


[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Download Med7 and spacy

*This is a temporary workaround to download med7 with the latest pip version*

In [2]:
!wget -O ./models/en_core_med7_lg-3.4.2.1-py3-none-any.whl https://huggingface.co/kormilitzin/en_core_med7_lg/resolve/main/en_core_med7_lg-any-py3-none-any.whl
%pip install ./models/en_core_med7_lg-3.4.2.1-py3-none-any.whl

--2024-08-29 15:52:26--  https://huggingface.co/kormilitzin/en_core_med7_lg/resolve/main/en_core_med7_lg-any-py3-none-any.whl
Resolving huggingface.co (huggingface.co)... 2600:9000:2553:200:17:b174:6d00:93a1, 2600:9000:2553:c00:17:b174:6d00:93a1, 2600:9000:2553:d000:17:b174:6d00:93a1, ...
Connecting to huggingface.co (huggingface.co)|2600:9000:2553:200:17:b174:6d00:93a1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/kormilitzin/en_core_med7_lg/2c8ce371b7ee8b3c42cce7c05031ab165e6e88cc3ed136797eb3c492af96001f?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27en_core_med7_lg-any-py3-none-any.whl%3B+filename%3D%22en_core_med7_lg-any-py3-none-any.whl%22%3B&Expires=1725202347&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNTIwMjM0N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9rb3JtaWxpdHppbi9lbl9jb3JlX21lZDdfbGcvMmM4Y2UzNzFiN2VlOGIzYzQyY2NlN2MwNTAzMWFiMTY1ZTZ

In [3]:
# Download spacy base model
%run -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 38.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')



[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


### Download example MedCAT models

In [4]:
# Create directories
MODEL_DIR = './models/'
DATA_DIR = './data/'

!mkdir $MODEL_DIR
!mkdir $DATA_DIR

mkdir: ./models/: File exists
mkdir: ./data/: File exists


In [5]:
%pip install gdown
import gdown


[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
gdown.download('https://drive.google.com/uc?id=11NZNd7k4J5t-wofDWsPxHMJ3Jg5QmbiI', MODEL_DIR + "miade_meds_model_a146c741501cf1f7.zip")
gdown.download('https://drive.google.com/uc?id=17s999FIotRenltR6gr_f8ZjdaXc-u1Gx', MODEL_DIR + 'miade_problems_model_f25ec9423958e8d6.zip')

Downloading...
From (original): https://drive.google.com/uc?id=11NZNd7k4J5t-wofDWsPxHMJ3Jg5QmbiI
From (redirected): https://drive.google.com/uc?id=11NZNd7k4J5t-wofDWsPxHMJ3Jg5QmbiI&confirm=t&uuid=166d9fc5-cd07-4427-a3b4-15abf102fe3c
To: /Users/jenniferjiangkells/Documents/miade-tutorials/notebooks/models/miade_meds_model_a146c741501cf1f7.zip
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 309M/309M [00:07<00:00, 39.8MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=17s999FIotRenltR6gr_f8ZjdaXc-u1Gx
From (redirected): https://drive.google.com/uc?id=17s999FIotRenltR6gr_f8ZjdaXc-u1Gx&confirm=t&uuid=12dd5faf-bda0-4824-be1b-48bfc630abc2
To: /Users/jenniferjiangkells/Documents/miade-tutorials/notebooks/models/miade_problems_model_f25ec9423958e8d6.zip
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 468M/468M [00:11<00:00, 42.4

'./models/miade_problems_model_f25ec9423958e8d6.zip'

### Download example data

In [7]:
!wget https://raw.githubusercontent.com/uclh-criu/miade-tutorials/master/notebooks/models/config.yaml -P $MODEL_DIR
!wget https://raw.githubusercontent.com/uclh-criu/miade-tutorials/master/notebooks/data/meds_allergies_cdb.csv -P $DATA_DIR
!wget https://raw.githubusercontent.com/uclh-criu/miade-tutorials/master/notebooks/data/wikipedia_sample.txt -P $DATA_DIR

--2024-08-29 15:53:16--  https://raw.githubusercontent.com/uclh-criu/miade-tutorials/master/notebooks/models/config.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 208 [text/plain]
Saving to: ‘./models/config.yaml.3’

config.yaml.3       100%[===================>]     208  --.-KB/s    in 0s      

2024-08-29 15:53:17 (10.4 MB/s) - ‘./models/config.yaml.3’ saved [208/208]

--2024-08-29 15:53:17--  https://raw.githubusercontent.com/uclh-criu/miade-tutorials/master/notebooks/data/meds_allergies_cdb.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
H

In [8]:
from pathlib import Path
from typing import List, Dict, Optional

from miade.core import NoteProcessor
from miade.note import Note
from miade.annotators import Annotator
from miade.concept import Concept, Category
from miade.dosageextractor import DosageExtractor

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/medcat/cat.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Part 1: Extracting concepts and dosages from notes using the full MiADE pipeline

### Configuring the MiADE processor

```NoteProcessor``` is the MiADE core. It is initialised with a model directory path that contains all the MedCAT model pack .zip files we would like to use in our pipeline, and a `config.yaml` file that maps an alias to the model IDs (model IDs can be found in MedCAT `model_cards` or usually will be in the name) and annotators we would like to use:

```
models:
  problems: f25ec9423958e8d6
  meds/allergies: a146c741501cf1f7
annotators:
  problems: ProblemsAnnotator
  meds/allergies: MedsAllergiesAnnotator
 ```

In [9]:
miade = NoteProcessor(Path(MODEL_DIR))

[2024-08-29 15:53:19,567] [INFO] miade.core._load_config(): Found config file models/config.yaml
[2024-08-29 15:53:19,569] [INFO] miade.core._load_model_factory(): Loading MedCAT models from models
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/medcat/meta_cat.py:356: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w

Once ```NoteProcessor``` is initialised, we can add annotators by the aliases we have specified in ```config.yaml``` to our processor. 

By default annotators will add negSpacy to MedCAT, which implements the negEx algorithm (Chapman et al. 2001) for negation detection. This allows the models to perform simple rule-based negation detection in the absence of trained MetaCAT models. You can disable this in the configurations if you wish to use your own MetaCAT instead.

In [10]:
miade.add_annotator("problems")

[2024-08-29 15:53:31,259] [INFO] miade.annotators._load_problems_lookup_data(): Loading preconfigured lookup data for ProblemsAnnotator
[2024-08-29 15:53:31,270] [INFO] miade.core.add_annotator(): Added ProblemsAnnotator to processor with config None


In [11]:
miade.add_annotator("meds/allergies")

[2024-08-29 15:53:31,284] [INFO] miade.annotators._load_med_allergy_lookup_data(): Loading preconfigured lookup data for MedsAllergiesAnnotator
[2024-08-29 15:53:31,489] [INFO] miade.core.add_annotator(): Added MedsAllergiesAnnotator to processor with config None


In [12]:
# print the MedCAT model cards to check that it's been mapped to the correct annotators
miade.print_model_cards()

ProblemsAnnotator: {
  "Model ID": "f25ec9423958e8d6",
  "Last Modified On": "12 June 2023",
  "History (from least to most recent)": [
    "0b130195e6964e66",
    "353abfe8f57e2009",
    "af37e525e37cdb1a"
  ],
  "Description": "MiADE problems development model unsupervised trained on MIMIC-III. cdb: full_condition_list_MedCAT_cdb.csv; vocab: medcat model 811218c0b819c304 (feb 2022) | Packaged with MetaCAT model(s) presence, relevance",
  "Source Ontology": "SNOMEDUK_May2022 MiADE subset",
  "Location": "gae03:gae/miade/models/problems/, miade-dev:vol/models/problems/",
  "MetaCAT models": [
    {
      "Category Name": "presence",
      "Description": "Blank meta model with tokenizer trained on MIMIC III",
      "Classes": {
        "confirmed": 0,
        "negated": 1,
        "suspected": 2
      },
      "Model": "lstm"
    },
    {
      "Category Name": "relevance",
      "Description": "Blank meta model with tokenizer trained on MIMIC III",
      "Classes": {
        "historic"

### Creating a Note

Next we will create a ```Note``` object which contains the text we would like to extract concepts and dosages from

In [13]:
text = """
Suspected heart failure

PMH:
prev history of hypothyroidism
MI 10 years ago


Current meds:
Losartan 100mg daily
Atorvastatin 20mg daily
Paracetamol 500mg tablets 2 tabs qds prn

Allergies:
Penicillin - rash

Referred with swollen ankles and shortness of breath since 2 weeks.
"""

In [14]:
note = Note(text)

### Extracting concepts and dosages

MiADE currently extracts concepts in SNOMED CT. Each concept has:
- `name`: name of concept
- `id`: concept ID
- `category`: type of concept e.g. problems, medictions
- `start`: start index of concept span
- `end`: end index of concept span
- `dosage`: for medication concepts
- `negex`: Negex result if configured
- `meta`: Meta annotations if MetaCAT models are used

The dosages associated with medication concepts are extracted by the built-in MiADE `DosageExtractor`, using a combination of NER model [Med7](https://github.com/kormilitzin/med7) and [the CALIBER rule-based drug dose lookup algorithm](https://rdrr.io/rforge/CALIBERdrugdose/). It returns:

- `dose`
- `duration`
- `frequency`
- `route`

The output format is directly translatable to HL7 CDA but can also easily be converted to FHIR.

In [15]:
# get concepts as dicts
concepts = miade.get_concept_dicts(note)

In [16]:
concepts

[{'name': 'heart failure (suspected)',
  'id': '394887005',
  'category': 'PROBLEM',
  'start': 11,
  'end': 24,
  'dosage': None,
  'linked_concepts': [],
  'negex': False,
  'meta': [{'name': 'presence',
    'value': 'SUSPECTED',
    'confidence': 0.9998881816864014},
   {'name': 'relevance', 'value': 'PRESENT', 'confidence': 0.999954104423523}],
  'debug': None},
 {'name': 'hypothyroidism',
  'id': '40930008',
  'category': 'PROBLEM',
  'start': 46,
  'end': 60,
  'dosage': None,
  'linked_concepts': [],
  'negex': False,
  'meta': [{'name': 'presence',
    'value': 'CONFIRMED',
    'confidence': 0.999984622001648},
   {'name': 'relevance',
    'value': 'HISTORIC',
    'confidence': 0.999841570854187}],
  'debug': None},
 {'name': 'swollen ankles',
  'id': '267039000',
  'category': 'PROBLEM',
  'start': 220,
  'end': 234,
  'dosage': None,
  'linked_concepts': [],
  'negex': False,
  'meta': [{'name': 'presence',
    'value': 'CONFIRMED',
    'confidence': 0.9856405258178711},
   {

#### Handling existing records - deduplication

MiADE is built to handle existing medication records from EHR systems that can be sent alongside the note. It will perform basic deduplication matching on id for existing record concepts.

In [17]:
# create list of concepts that already exists in patient record
record_concepts = [
    Concept(id="40930008", name="hypothyroidism", category=Category.PROBLEM),
    Concept(id="267039000", name="swollen ankle", category=Category.PROBLEM)
]

In [18]:
miade.get_concept_dicts(note=note, record_concepts=record_concepts)

[{'name': 'heart failure (suspected)',
  'id': '394887005',
  'category': 'PROBLEM',
  'start': 11,
  'end': 24,
  'dosage': None,
  'linked_concepts': [],
  'negex': False,
  'meta': [{'name': 'presence',
    'value': 'SUSPECTED',
    'confidence': 0.9998881816864014},
   {'name': 'relevance', 'value': 'PRESENT', 'confidence': 0.999954104423523}],
  'debug': None},
 {'name': 'shortness of breath',
  'id': '267036007',
  'category': 'PROBLEM',
  'start': 238,
  'end': 257,
  'dosage': None,
  'linked_concepts': [],
  'negex': False,
  'meta': [{'name': 'presence',
    'value': 'CONFIRMED',
    'confidence': 0.9906362891197205},
   {'name': 'relevance',
    'value': 'PRESENT',
    'confidence': 0.9999895095825195}],
  'debug': None},
 {'name': 'LOSARTAN ORAL',
  'id': None,
  'category': 'MEDICATION',
  'start': 91,
  'end': 99,
  'dosage': {'dose': {'source': '100 mg',
    'value': 100.0,
    'unit': 'mg',
    'low': None,
    'high': None},
   'duration': None,
   'frequency': {'sourc

#### Removing annotators

We can also remove annotators if no longer needed

In [19]:
miade.remove_annotator("meds/allergies")

[2024-08-29 15:53:31,743] [INFO] miade.core.remove_annotator(): Removed MedsAllergiesAnnotator from processor


In [20]:
miade.get_concept_dicts(note=note, record_concepts=record_concepts)

[{'name': 'heart failure (suspected)',
  'id': '394887005',
  'category': 'PROBLEM',
  'start': 11,
  'end': 24,
  'dosage': None,
  'linked_concepts': [],
  'negex': False,
  'meta': [{'name': 'presence',
    'value': 'SUSPECTED',
    'confidence': 0.9998881816864014},
   {'name': 'relevance', 'value': 'PRESENT', 'confidence': 0.999954104423523}],
  'debug': None},
 {'name': 'shortness of breath',
  'id': '267036007',
  'category': 'PROBLEM',
  'start': 238,
  'end': 257,
  'dosage': None,
  'linked_concepts': [],
  'negex': False,
  'meta': [{'name': 'presence',
    'value': 'CONFIRMED',
    'confidence': 0.9906362891197205},
   {'name': 'relevance',
    'value': 'PRESENT',
    'confidence': 0.9999895095825195}],
  'debug': None}]

## Part 2: Customising MiADE

### 2.1 Building custom MedCAT models

This part of the tutorial will go through the basics of building a MedCAT model that can be used with MiADE.

For more information on **MedCAT**:

- [Github](https://github.com/CogStack/MedCAT)
- [MedCAT Tutorials](https://github.com/CogStack/MedCATtutorials)
- [Paper](https://arxiv.org/abs/2010.01165)

In [21]:
from miade.model_builders.cdbbuilder import CDBBuilder
from miade.model_builders.vocabbuilder import VocabBuilder

In [22]:
CDB_DATA_PATH = DATA_DIR + "meds_allergies_cdb.csv"

In [23]:
TEXT_DATA_PATH = DATA_DIR + "wikipedia_sample.txt"

#### **Create a CDB (concept databse) builder**

The MiADE `CDBBuilder` uses SNOMED CT and builds from a preprocessed CSV in the following example format:

| cui      | name                  | ontologies | name_status |
| -------- | --------------------- | ---------- | ----------- |
| 59927004 | hepatic failure       | SNO        | P           |
| 59927004 | hepatic insufficiency | SNO        | A           |


Where "P" denotes primary name and "A" denotes alternate name. 

See MedCAT Tutorials For more information on building CDB and Vocabs with custom ontologies

In [24]:
cdbbuilder = CDBBuilder(
    temp_dir=(Path.cwd() / Path(".temp")),  # temp dir for saving data
    custom_data_paths=[Path(CDB_DATA_PATH)]
)

In [25]:
# process and create the CDB object
cdbbuilder.preprocess()
cdb = cdbbuilder.create_cdb()

In [26]:
# inspect the vocab
cdb.vocab

{'acetaminophen': 1,
 '500mg': 3,
 'oral': 4,
 'tablet': 4,
 'paracetamol': 1,
 'ibuprofen': 1,
 '200mg': 1,
 'penicillin': 1,
 'coconut': 1,
 'metformin': 1,
 'hydrochloride': 1,
 'peanut': 1,
 'butter': 1,
 'a': 2,
 'rash': 2,
 'complaining': 1,
 'of': 3,
 'facial': 1,
 'swelling': 1,
 'symptoms': 2,
 'depression': 2,
 'symptom': 2,
 'depressive': 2}

#### **Create a Vocab builder**

The `Vocab` takes a free text as input and trains a corpus of word embeddings on it using [Word2Vec](https://www.tensorflow.org/tutorials/text/word2vec). 

Using `VocabBuilder` we can pass a list of texts we would like to create our vocab with - here we are using some wikipedia sample text.

In [27]:
# load the training data in a list
with open(TEXT_DATA_PATH, "r", encoding="utf-8") as training_data:
    training_data_list = [line.strip() for line in training_data]

In [28]:
# inspect the data
training_data_list

['Lung cancer, also known as bronchial carcinoma, since about 98–99% of all lung cancers are carcinomas, is a malignant lung tumor characterized by uncontrolled cell growth in tissues of the lung. Lung carcinomas derive from transformed, malignant cells that originate as epithelial cells, or from tissues composed of epithelial cells. Other lung cancers, such as the rare sarcomas of the lung, are generated by the malignant transformation of connective tissues (i.e. nerve, fat, muscle, bone), which arise from mesenchymal cells. Lymphomas and melanomas (from lymphoid and melanocyte cell lineages) can also rarely result in lung cancer.',
 'In time, this uncontrolled growth can spread beyond the lung – either by direct extension, by entering the lymphatic circulation, or via the hematogenous, bloodborne spread – the process called metastasis – into nearby tissue or other, more distant parts of the body. Most cancers that start in the lung, known as primary lung cancers, are carcinomas. The 

In [29]:
# create a builder
vocab_builder = VocabBuilder()

In [30]:
# create the new vocab
vocab = vocab_builder.create_new_vocab(
    training_data_list=training_data_list,
    cdb=cdb,
    config=cdb.config
)

In [31]:
# inspect the vocab
vocab.vocab.keys()

dict_keys(['lung', 'cancer', 'also', 'known', 'as', 'bronchial', 'carcinoma', 'since', 'about', '98', '99', 'of', 'all', 'cancers', 'are', 'carcinomas', 'is', 'a', 'malignant', 'tumor', 'characterized', 'by', 'uncontrolled', 'cell', 'growth', 'in', 'tissues', 'the', 'derive', 'from', 'transformed', 'cells', 'that', 'originate', 'epithelial', 'or', 'composed', 'other', 'such', 'rare', 'sarcomas', 'generated', 'transformation', 'connective', 'i', 'e', 'nerve', 'fat', 'muscle', 'bone', 'which', 'arise', 'mesenchymal', 'lymphomas', 'and', 'melanomas', 'lymphoid', 'melanocyte', 'lineages', 'can', 'rarely', 'result', 'time', 'this', 'spread', 'beyond', 'either', 'direct', 'extension', 'entering', 'lymphatic', 'circulation', 'via', 'hematogenous', 'bloodborne', 'process', 'called', 'metastasis', 'into', 'nearby', 'tissue', 'more', 'distant', 'parts', 'body', 'most', 'start', 'primary', 'two', 'main', 'types', 'small', 'sclc', 'non', 'nsclc', 'common', 'symptoms', 'coughing', 'including', 'up'

#### **Create a model pack**

In [32]:
# saves the model pack into .zip with the vocab and cdb we just created - this is what we load into miade
vocab_builder.make_model_pack(cdb=cdb, save_name="miade_example")

#### Run training from script (best used in command line)

MiADE provides scripts for automatically building MedCAT model packs, unsupervised training, supervised training steps, and the creation and training of MetaCAT models. 

The ```--synthetic-data-path``` option allows you to add synthetically generated training data in CSV format to the supervised and MetaCAT training steps. The CSV should have the following format:

| text                          | cui               | name                       | start | end | relevance | presence  | laterality |
| ----------------------------- | ----------------- | -------------------------- | ----- | --- | --------- | --------- | -------------------- |
| no history of liver failure | 59927004 | hepatic failure      | 14     | 26  | historic  | negated | none                 

In [ ]:
# Path to MedCAT model pack to use for training
MODEL_PACK_PATH = ""

# Path to MedCATTrainer JSON export (annotations)
MEDCAT_JSON_EXPORT = ""

# Path to synthetic data in CSV format
SYNTHETIC_CSV_PATH = ""

# Path to the tokenizer to use for MetaCAT model training
TOKENIZER_PATH = ""

# List of category names for MetaCAT e.g. presence, relevance - will create MetaCAT folders in the format "meta_$CATEGORY_NAME"
CATEGORY_NAMES = []

# Path to the MetaCAT model folders to train
METACAT_MODL_PATH = ""

In [ ]:
# Trains unsupervised training step of MedCAT model
!miade train $MODEL_PACK_PATH $TEXT_DATA_PATH --tag "miade-example"

In [ ]:
# Trains supervised training step of MedCAT model
!miade train-supervised $MODEL_PACK_PATH $MEDCAT_JSON_EXPORT --synthetic-data-path $SYNTHETIC_CSV_PATH

In [ ]:
# Creates BBPE tokenizer for MetaCAT
!miade create-bbpe-tokenizer $TEXT_DATA_PATH

In [ ]:
# Initialises MetaCAT models to do training on
!miade create-metacats $TOKENIZER_PATH $CATEGORY_NAMES

In [ ]:
# Trains the MetaCAT Bi-LSTM models
!miade train-metacats $METACAT_MODEL_PATH $MEDCAT_JSON_EXPORT --synthetic-data-path $SYNTHETIC_CSV_PATH

In [ ]:
# Packages MetaCAT models with the main MedCAT model pack
!miade add_metacat_models $MODEL_PACK_PATH $METACAT_MODEL_PATH

#### **And more...**

[**MiADE training dashboard**](https://github.com/uclh-criu/miade/tree/master/streamlit_app): A streamlit app that allows you to interactively inspect synthetic data and train MetaCAT models

### 2.2 Creating custom MiADE annotators

We can add custom annotators with more specialised postprocessing steps to MiADE by subclassing Annotator and initialising NoteProcessor with a list of custom annotators.

Built-in Annotator pipeline methods include:

`preprocess`: performs basic text cleaning and structural information on the note
`medcat`: returns MedCAT output as MiADE Concepts
`dosage_extractor`: uses the MiADE built-in DosageExtractor to add dosages associated with medication concepts
`deduplicator`: filters duplicate concepts in list

You must specify the type of concepts your custom annotator returns (see Category), a pipeline processing order, and implement a postprocess() function. An example custom Annotator class might look like this:

In [33]:
from miade.utils.miade_cat import CAT
from miade.utils.annotatorconfig import AnnotatorConfig


class CustomAnnotator(Annotator):
    def __init__(self, cat: CAT, config: AnnotatorConfig):
        super().__init__(cat, config)
        self.reactions = ["271807003"]
        self.allergens = ["764146007"]

    @property
    def concept_types(self) -> List[Category]:
        return [Category.MEDICATION, Category.ALLERGY]

    @property
    def pipeline(self) -> List[str]:
        return ["preprocessor", "medcat", "postprocessor", "dosage_extractor", "deduplicator"]

    def postprocess(self, concepts: List[Concept]) -> List[Concept]:
        # some example post-processing code
        for concept in concepts:
            if concept.id in self.reactions:
                concept.category = Category.REACTION
            elif concept.id in self.allergens:
                concept.category = Category.ALLERGY
        return concepts

Don't forget to add the custom annotator to `config.yaml`
```
models:
  problems: f25ec9423958e8d6
  meds/allergies: a146c741501cf1f7
  custom: a146c741501cf1f7
annotators:
  problems: ProblemsAnnotator
  meds/allergies: MedsAllergiesAnnotator
  custom: CustomAnnotator
 ```

In [34]:
# pass custom annotator to miade
miade = NoteProcessor(Path(MODEL_DIR), custom_annotators=[CustomAnnotator])

[2024-08-29 15:53:42,790] [INFO] miade.core._load_config(): Found config file models/config.yaml
[2024-08-29 15:53:42,792] [INFO] miade.core._load_model_factory(): Loading MedCAT models from models
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/medcat/meta_cat.py:356: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w

In [35]:
miade.add_annotator("problems")

[2024-08-29 15:53:57,671] [INFO] miade.annotators._load_problems_lookup_data(): Loading preconfigured lookup data for ProblemsAnnotator
[2024-08-29 15:53:57,681] [INFO] miade.core.add_annotator(): Added ProblemsAnnotator to processor with config None


In [36]:
miade.add_annotator("custom")

[2024-08-29 15:53:57,689] [INFO] miade.core.add_annotator(): Added CustomAnnotator to processor with config None


In [37]:
miade.get_concept_dicts(note)

[{'name': 'heart failure (suspected)',
  'id': '394887005',
  'category': 'PROBLEM',
  'start': 11,
  'end': 24,
  'dosage': None,
  'linked_concepts': [],
  'negex': False,
  'meta': [{'name': 'presence',
    'value': 'SUSPECTED',
    'confidence': 0.9998881816864014},
   {'name': 'relevance', 'value': 'PRESENT', 'confidence': 0.999954104423523}],
  'debug': None},
 {'name': 'hypothyroidism',
  'id': '40930008',
  'category': 'PROBLEM',
  'start': 46,
  'end': 60,
  'dosage': None,
  'linked_concepts': [],
  'negex': False,
  'meta': [{'name': 'presence',
    'value': 'CONFIRMED',
    'confidence': 0.999984622001648},
   {'name': 'relevance',
    'value': 'HISTORIC',
    'confidence': 0.999841570854187}],
  'debug': None},
 {'name': 'swollen ankles',
  'id': '267039000',
  'category': 'PROBLEM',
  'start': 219,
  'end': 233,
  'dosage': None,
  'linked_concepts': [],
  'negex': False,
  'meta': [{'name': 'presence',
    'value': 'CONFIRMED',
    'confidence': 0.9856405258178711},
   {

In [38]:
# or we can use .process() to get the Concept objects directly
concepts = miade.process(note)
for concept in concepts:
    if concept.category == Category.ALLERGY or concept.category == Category.REACTION:
        print(concept)

{name: Penicillin, id: 764146007, category: Category.ALLERGY, start: 188, end: 198, dosage: None, linked_concepts: [], negex: False, meta: None} 
{name: rash, id: 271807003, category: Category.REACTION, start: 199, end: 203, dosage: None, linked_concepts: [], negex: False, meta: None} 


### 3.3 Configurations

For more information on MiADE configurations, including using custom lookup tables for postprocessing, see [Documentation page](https://uclh-criu.github.io/miade/user-guide/configurations/)

## End of tutorial
